# Loading the reference images

In [ ]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from cloth_tools.dataset.format import load_competition_observation

data_dir = Path("data")
references_dir = data_dir / "cloth_competition_references_0002"
teams_dir = data_dir / "evaluation_icra_2024"
grasp_success_csv = data_dir / "evaluation_icra_2024_grasp_success.csv"

In [ ]:
os.path.exists(references_dir)

In [ ]:
reference_sample_dirs = [references_dir / dirname for dirname in sorted(os.listdir(references_dir))]
reference_sample_dirs

In [ ]:
reference_observation_dirs = [sample_dir / "observation_result" for sample_dir in reference_sample_dirs]
# reference_observation_dirs = reference_sample_dirs # FOR THE OLD REFERENCE FOLDER STRUCTURE

# observation_dirs = [dataset_dir / ref_dir for ref_dir in sorted(os.listdir(dataset_dir))]
len(reference_observation_dirs)

In [ ]:
import json

reference_images = {}
reference_surface_areas = {}

for cloth_id, observation_dir in enumerate(reference_observation_dirs):
    observation = load_competition_observation(observation_dir)

    results_file = observation_dir / "result.json"

    if results_file.exists():
        with open(results_file, "r") as f:
            result = json.load(f)
        surface_area = result["coverage"]
        # reference_images[f"Evaluation reference {cloth_id} ({surface_area:.3f} m²)"] = observation.image_left
        reference_images[f"Evaluation reference {cloth_id}"] = observation.image_left
        reference_surface_areas[cloth_id] = surface_area
    else:
        reference_images[f"Train reference {cloth_id}"] = observation.image_left


In [ ]:
cloth_id = 0
image = reference_images[list(reference_images.keys())[0]]

print(image.shape)



image_cropped = image[20:1020, 660:1660, :] # for references at ICRA
# image_cropped = image[20:1020, 620:1620, :] 

print(image_cropped.shape)

plt.imshow(image_cropped)

In [ ]:
def sample_id_to_reference_id(sample_id):
    return sample_id % 8

def reference_id_to_cloth_type(reference_id):
    if reference_id in [0,1,4,5]:
        return "shirt"
    else:
        return "towel"
    
def reference_id_to_seen(reference_id):
    if reference_id in [0,2,4,6]:
        return True
    else:
        return False

sample_id_to_reference_id(8), sample_id_to_reference_id(10)

In [ ]:
data_dicts = []

team_names = ["AIR_JNU", "Team_Ljubljana", "Ewha_Glab", "SCUT-ROBOT", "test_greater_bay", "SRCB-Test", "Shibata_Lab", "ai_robot", "UOS_robotics", "AIS_team", "3c1s"]

for team_name in team_names:
    team_dir = teams_dir / team_name
    team_sample_dirs = [team_dir / dirname for dirname in sorted(os.listdir(team_dir)) if dirname.startswith("sample_")]

    for i, sample_dir in enumerate(team_sample_dirs):
        
        observation_results_dir = sample_dir / "observation_result"
        
        # Load the surface are
        results_file = observation_results_dir / "result.json"

        with open(results_file, "r") as f:
            result = json.load(f)

        reference_id = sample_id_to_reference_id(i)
        surface_area = result["coverage"]
        reference_surface_area = reference_surface_areas[reference_id]
        coverage = surface_area / reference_surface_area

        data_dict = {
            "team_name": team_name,
            "sample_id": i,
            "reference_id": sample_id_to_reference_id(i),
            "sample_dir": sample_dir,
            "surface_area": surface_area,
            "coverage": coverage,
            "cloth_type": reference_id_to_cloth_type(reference_id),
            "seen": reference_id_to_seen(reference_id),
        }

        data_dicts.append(data_dict)

# convert to pandas dataframe
import pandas as pd

df = pd.DataFrame(data_dicts)

df

In [ ]:
# Define the function to sort coverage for a given reference_id
def sort_coverage(ref_id):
    """
    Sorts the coverage values for a given reference_id in ascending order.

    Args:
      ref_id: The reference_id to filter and sort by.

    Returns:
      A DataFrame with rows matching the reference_id, sorted by coverage.
    """
    filtered_df = df[df["reference_id"] == ref_id]
    sorted_df = filtered_df.sort_values(by="coverage", ascending=True)
    return sorted_df

# Call the function with reference_id = 2
sorted_coverage_df = sort_coverage(ref_id=2)
sorted_coverage_df

In [ ]:
for index, row in sorted_coverage_df.iterrows():
  print(row['coverage'], row['sample_dir']) 
  sample_dir = row['sample_dir']

  observation = load_competition_observation(observation_dir)
  image = observation.image_left

In [ ]:
import matplotlib.pyplot as plt

def sort_coverage(df, reference_id):
  """
  Sorts the dataframe by coverage for a given reference_id.

  Args:
    df: The pandas DataFrame.
    reference_id: The reference_id to filter and sort by.

  Returns:
    A new DataFrame sorted by coverage in ascending order, with original index preserved.
  """
  df_filtered = df[df['reference_id'] == reference_id]
  df_sorted = df_filtered.sort_values('coverage', ascending=True)
  return df_sorted


def plot_images_by_coverage(df, ref_id):
  """
  Plots a grid of images sorted by coverage for a given reference_id.

  Args:
    df: The pandas DataFrame with 'reference_id', 'coverage', and 'sample_dir' columns.
    ref_id: The reference ID to filter and plot images for.
  """

  sorted_df = sort_coverage(df, ref_id)

  num_images = len(sorted_df)
  cols = 4  # You can adjust the number of columns as needed
  rows = int(num_images / cols) + (num_images % cols > 0)

  fig, axes = plt.subplots(rows, cols, figsize=(16, 3 * rows))
  axes = axes.flatten()

  for i, (index, row) in enumerate(sorted_df.iterrows()):
    coverage = row['coverage']
    sample_dir = row['sample_dir']
    observation_dir = sample_dir / "observation_result"

    observation = load_competition_observation(observation_dir)
    image = observation.image_left

    # Include the original row number in the title
    axes[i].imshow(image)
    axes[i].set_title(f"Row: {index}, Coverage: {coverage:.2f}")
    axes[i].axis('off')

  for i in range(num_images, len(axes)):
    axes[i].axis('off')

  plt.tight_layout()
  plt.show()

# Example usage:
plot_images_by_coverage(df, 0)

In [ ]:
plot_images_by_coverage(df, 2)

In [ ]:
import matplotlib.pyplot as plt

def plot_images_by_row_numbers(df, row_numbers):
  """
  Plots a 2x5 grid of images from the DataFrame based on specified row numbers.

  Args:
    df: The pandas DataFrame with 'coverage', and 'sample_dir' columns.
    row_numbers: A list of row numbers from the original DataFrame to plot.
  """

  fig, axes = plt.subplots(2, 5, figsize=(15, 5.9), dpi=300)  # 2x5 grid
  axes = axes.flatten()

  for i, row_num in enumerate(row_numbers):
    row = df.iloc[row_num]  # Get the row by its original index
    coverage = row['coverage']
    sample_dir = row['sample_dir']
    observation_dir = sample_dir / "observation_result"

    observation = load_competition_observation(observation_dir)
    image = observation.image_left

    # image_cropped = image[20:1020, 660:1660, :]
    # image_cropped = image[20:1020, 610:1710, :]
    image_cropped = image[20:1020, 560:1760, :]

    axes[i].imshow(image_cropped)
    axes[i].set_title(f"Coverage: {coverage:.2f}")
    axes[i].axis('off')

  plt.tight_layout(pad=0.1)
  plt.show()

# Example usage:
row_numbers_to_plot = [152, 160, 104, 72, 8, 98, 42, 66, 58, 26]
plot_images_by_row_numbers(df, row_numbers_to_plot)